In [36]:
! pip install huggingface_hub
! git config --global credential.helper store

In [37]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder
import pandas as pd



In [39]:
#enter your API key, you can make one for free on HF
notebook_login()  

Login successful
Your token has been saved to /Users/siennaaa/.huggingface/token


In [40]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [41]:
exp_condition_df = pd.read_csv('../experiments/experiment2/tasks_collection_essentialism_exp2.csv')
exp_condition_df = exp_condition_df.fillna("")
exp_condition_df


,category,version_number,condition_description,items,item_original,item_new,appearance_type,whose_appearance,comp_condition,whose_comp,scenario_1,scenario_2,question,question_mask
0,non_living_natural_kinds,nl_1_lc_1,lightning_to_cloud_app_original_comp_preserved,lightning_to_cloud,lightning,cloud,original,lightning,preserved,lightning,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
1,non_living_natural_kinds,nl_1_lc_2,lightning_to_cloud_app_original_comp_changed,lightning_to_cloud,lightning,cloud,original,lightning,changed,cloud,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
2,non_living_natural_kinds,nl_1_lc_3,lightning_to_cloud_app_new_comp_preserved,lightning_to_cloud,lightning,cloud,new,cloud,preserved,lightning,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
3,non_living_natural_kinds,nl_1_lc_4,lightning_to_cloud_app_new_comp_changed,lightning_to_cloud,lightning,cloud,new,cloud,changed,cloud,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
4,non_living_natural_kinds,nl_2_lsun_1,lightning_to_sun_app_original_comp_preserved,lightning_to_sun,lightning,sun,original,lightning,preserved,lightning,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,artifacts,a_11_km_4,keychain_to_microwave_app_new_comp_changed,keychain_to_microwave,keychain,microwave,new,microwave,changed,microwave,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is
140,artifacts,a_12_kl_1,keychain_to_lotion_app_original_comp_preserved,keychain_to_lotion,keychain,lotion,original,keychain,preserved,keychain,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is
141,artifacts,a_12_kl_2,keychain_to_lotion_app_original_comp_changed,keychain_to_lotion,keychain,lotion,original,keychain,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is
142,artifacts,a_12_kl_3,keychain_to_lotion_app_new_comp_preserved,keychain_to_lotion,keychain,lotion,new,lotion,preserved,keychain,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is


In [53]:
def bloom_infer(prompt, seed=2022):
    # top_k = None if top_k == 0 else top_k
    # do_sample = False if num_beams > 0 else not greedy_decoding
    # num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    # no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    # top_p = None if num_beams else top_p
    # early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": 5,
        "top_k": None,
        "top_p": 0.9,
        "temperature": 0.7,
        "do_sample": True,
        "seed": seed,
        "early_stopping": False,
        "no_repeat_ngram_size": None,
        "num_beams": None,
        "eos_token_id": None,
        "length_penalty": 1.0,
    }
    

    response = inference(prompt, params=params)

    return response

In [54]:

bloom_response_df = pd.DataFrame(columns=exp_condition_df.columns)

for exp_condition in range(...,...): #range(2): #range[1] means the second row of the csv file, can change to range[2] to get the third row of the csv file. We will want to start with [1] since we ran the first row already.
    for rep_id in range(...): 
      exp_row = exp_condition_df.loc[[exp_condition]]
      exp_row["rep_id"] = rep_id + 1

      prompt = ""
      for scenario_id in range(1, 3): #range(1, 3):  # We have two scenarios.
          scenario_column_name = f"scenario_{scenario_id}"  # f"scenario_{scenario_id}" is scenario column name
        #   print(exp_condition_df[scenario_column_name][exp_condition])


          prompt = ""
          prompt += "\n\nQuestion: " + exp_condition_df[scenario_column_name][exp_condition].strip() + " " + exp_condition_df["question"][exp_condition] + "\nAnswer:" + " I think the thing after the operation is "

          print(prompt, end="")
          response = bloom_infer(prompt, seed=rep_id)
          answer = response[0]["generated_text"][len(prompt):].strip()
          print(answer)
          print("-------------------------------------------------------------------\n")


          bloom_response_column_name = f"bloom_response_scenario_{scenario_id}" 
          exp_row[bloom_response_column_name] = answer

          prompt += answer + "\n"

      bloom_response_df = pd.concat([bloom_response_df, exp_row], axis=0)

bloom_response_df.rep_id = bloom_response_df.rep_id.astype(int)

col = bloom_response_df.pop("rep_id")
bloom_response_df.insert(2, col.name, col)

bloom_response_df





Question: Some very talented and skilled scientists decide that they are going to perform a special procedure to turn a bed into a microwave. After the special procedure, the thing looked like a bed. After running some tests, they found that the thing after the special procedure wasn’t made of metal, plastic, glass, and ceramic components. Instead, it was only made of wood, metal, or upholstered fabric. Is the thing after the special operation a bed or a microwave?
Answer: I think the thing after the operation is not a microwave, because a
-------------------------------------------------------------------



Question: Some very talented and skilled scientists decide that they are going to perform a special procedure to turn a bed into a microwave. After running some tests, they found that the thing after the special procedure wasn’t made of metal, plastic, glass, and ceramic components. Instead, it was only made of wood, metal, or upholstered fabric. After the special procedure, the

,category,version_number,rep_id,condition_description,items,item_original,item_new,appearance_type,whose_appearance,comp_condition,whose_comp,scenario_1,scenario_2,question,question_mask,bloom_response_question_1,bloom_response_question_2
96,artifacts,a_1_bm_1,1,bed_to_microwave_app_original_comp_preserved,bed_to_microwave,bed,microwave,original,bed,preserved,bed,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a bed...,The thing after the special operation is,"not a microwave, because a",a microwave. Because in the
96,artifacts,a_1_bm_1,2,bed_to_microwave_app_original_comp_preserved,bed_to_microwave,bed,microwave,original,bed,preserved,bed,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a bed...,The thing after the special operation is,a microwave. The reason is,a microwave because if you put
96,artifacts,a_1_bm_1,3,bed_to_microwave_app_original_comp_preserved,bed_to_microwave,bed,microwave,original,bed,preserved,bed,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a bed...,The thing after the special operation is,a microwave. After the special,a microwave. Because I think
96,artifacts,a_1_bm_1,4,bed_to_microwave_app_original_comp_preserved,bed_to_microwave,bed,microwave,original,bed,preserved,bed,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a bed...,The thing after the special operation is,a microwave. I think this,"a microwave, not a bed"
96,artifacts,a_1_bm_1,5,bed_to_microwave_app_original_comp_preserved,bed_to_microwave,bed,microwave,original,bed,preserved,bed,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a bed...,The thing after the special operation is,a microwave. The answer is,a microwave because it can cook
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,artifacts,a_12_kl_4,1,keychain_to_lotion_app_new_comp_changed,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,a keychain. Because the,keychain because the thing before
143,artifacts,a_12_kl_4,2,keychain_to_lotion_app_new_comp_changed,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,both a keychain and lot,"a lotion, because the"
143,artifacts,a_12_kl_4,3,keychain_to_lotion_app_new_comp_changed,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,a keychain because it looks,a keychain.\nWhy?
143,artifacts,a_12_kl_4,4,keychain_to_lotion_app_new_comp_changed,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,a keychain. According to,"lotion, because it looks"


In [55]:
print("\n==========================\n".join(bloom_response_df["bloom_response_question_1"].tolist()))


not a microwave, because a
a microwave. The reason is
a microwave. After the special
a microwave. I think this
a microwave. The answer is
a microwave. The reason is
a microwave because the procedure they
a microwave. After all,
a microwave. In my opinion
a microwave because it can be
a microwave. Because if it
a microwave because the microwave is
a microwave.

A:

I
both a bed and a microwave
a microwave. The reason is
a microwave.

A:

The
both a bed and a microwave
a microwave, because the scientists
a microwave, because it looks
a microwave, because the operation
a bed, not lotion
a bed. It was made
a bed. Because I think
not a bed, because it
a bed. If they used
a bed. Because, if
a lotion because it is
a bed. I think the
a bed. It is because
a bed. Because after the
a bed because a bed is
a bed. Because, in
lotion. Because the test
a bed, because the scientists
a bed.
We know that
a bed, because it is
lotion.
Question: I am
lotion. Because the thing
a bed, because after the
lotion

In [56]:
bloom_response_df.to_csv("../../data/.../.../....csv")